# MottanAI: Garbage Classifier using CNN

### Import Libraries

In [1]:
import os
import pathlib
import numpy as np
from PIL import Image, ImageOps
from imageio import imread, imwrite
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers



### Global Constants

In [1]:
img_height = 180
img_width = 180

### Preprocessing

In [ ]:
def preprocess_dataset():
    """ preprocess gathered images into burnable and nonburnable folders """

    dataset_original_path = os.path.join(os.getcwd(), 'dataset_original')
    dataset_output_path = os.path.join(os.getcwd(), 'dataset')

    try:
        os.makedirs(dataset_output_path)
        os.makedirs(os.path.join(dataset_output_path, 'burnable'))
        os.makedirs(os.path.join(dataset_output_path, 'nonburnable'))
        
    except OSError:
        if not os.path.isdir(dataset_output_path):
            raise

    for original_class in is_burnable:

        original_class_path = os.path.join(dataset_original_path,
                                           original_class)

        preprocessed_class_path = os.path.join(
            dataset_output_path,
            'burnable' if is_burnable[original_class] else 'nonburnable')

        for subdir, dirs, files in os.walk(original_class_path):
        for file in files:
            if len(file) <= 4 or file[-4:] != '.jpg':
                continue

            image = Image.open(os.path.join(subdir, file))

            # Apply Grayscale
            # image = ImageOps.grayscale(image)

            # Resize image
            image = image.resize((img_width, img_height))

            imwrite(os.path.join(preprocessed_class_path, file), image)

# uncomment only if new images are added to the original dataset         
#preprocess_dataset()

### Load the Dataset

In [ ]:

# Parameters
batch_size = 32
validation_split = 0.3  # Train-test split. Validation = Test

print('Importing dataset...')
data_dir = pathlib.Path('dataset')

# Get the number of images
image_count = len(list(data_dir.glob('*/*.jpg')))
print(f"image_count = {image_count}")

#
train_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="training",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

test_ds = tf.keras.utils.image_dataset_from_directory(
    data_dir,
    validation_split=validation_split,
    subset="validation",
    seed=123,
    image_size=(img_height, img_width),
    batch_size=batch_size)

print('Found classes ', train_ds.class_names)

# Use buffered prefetching
print('Using buffered prefetching...')
AUTOTUNE = tf.data.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)